In [ ]:
import numpy as np
import pandas as pd 
import time
import datetime
import requests
import lxml
from lxml.html import fromstring
from fake_useragent import UserAgent


ua = UserAgent()
headers = {'User-Agent' : ua.random}

# date 대체용: 오늘 월,일 정의 
today_day = '%02d' % (datetime.datetime.now().day)
today_month = '%02d' % (datetime.datetime.now().month)
today_date = f'{today_month}/{today_day}'

# 데이터프레임 초기화

In [ ]:
df = pd.DataFrame(columns=['제목','글쓴이','게시판','등록일','추천수','조회수','url'])

# URL 크롤링 (page만 변경할 것)

In [ ]:
start = time.time()
page = 1 ######### 수정 수집 시 여기만 변경
date = today_date

while True:
    
    # page 바꾸기
    base_url = f'https://www.bobaedream.co.kr/list?code=best&s_cate=&maker_no=&model_no=&or_gu=10&or_se=desc&s_selday=&pagescale=30&info3=&noticeShow=&s_select=&s_key=&level_no=&bestCode=&bestDays=&bestbbs=&vdate=&type=list&page={page}'
    
    try:
        res = requests.get(base_url, headers=headers)
        res.raise_for_status()
        res.encoding = "utf-8-sig"
        parser = fromstring(res.text)
        
    except:
        
        try:
            print(f'{page}페이지 에러발생!!! 5초를 쉽니다.')
            time.sleep(5)
            res = requests.get(base_url, headers=headers)
            res.raise_for_status()
            res.encoding = "utf-8-sig"
            parser = fromstring(res.text)
        
        except:
            
            try:
                print(f'{page}페이지 2차 에러발생!!! 5초를 쉽니다.')
                time.sleep(5)
                res = requests.get(base_url, headers=headers)
                res.raise_for_status()
                res.encoding = "utf-8-sig"
                parser = fromstring(res.text)
                
            except:
                print(f'{page}페이지 3차 에러발생!!! 5초를 쉽니다.')
                time.sleep(5)
                res = requests.get(base_url, headers=headers)
                res.raise_for_status()
                res.encoding = "utf-8-sig"
                parser = fromstring(res.text)
            
        

    print(f'== {page}페이지 시작 == 경과시간: {round(time.time() - start)}초 == 현재 날짜: {date}')


    for num in range(1, 31):

        # find element
        title = parser.xpath(f'//*[@id="boardlist"]/tbody/tr[{num}]/td[2]/a[1]')[0].text_content()
        author = parser.xpath(f'//*[@id="boardlist"]/tbody/tr[{num}]/td[3]/span[2]')[0].text_content()
        board = parser.xpath(f'//*[@id="boardlist"]/tbody/tr[{num}]/td[1]')[0].text_content()
        date = parser.xpath(f'//*[@id="boardlist"]/tbody/tr[{num}]/td[4]')[0].text_content()
        if ':' in date:
            date = today_date
        view = parser.xpath(f'//*[@id="boardlist"]/tbody/tr[{num}]/td[6]')[0].text_content()
        recommend = parser.xpath(f'//*[@id="boardlist"]/tbody/tr[{num}]/td[5]/font[1]')[0].text_content()
        url = 'https://www.bobaedream.co.kr'+parser.xpath(f'//*[@id="boardlist"]/tbody/tr[{num}]/td[2]/a[1]')[0].attrib['href']

        # append df
        tmp = pd.DataFrame([[title, author, board, date, recommend, view, url]], columns = ['제목','글쓴이','게시판','등록일','추천수','조회수','url'])
        df = df.append(tmp, ignore_index=True)
        
    # add page
    page += 1
    
    # stopping condition
    if date[:2] == '10': # 6개월치만 수집
        print('수집종료')
        break
        

# 본문 크롤링

In [ ]:
df

In [ ]:
df['내용'] = 0
start = time.time()

for idx in range(df.shape[0]):
    
    url = df.loc[idx, 'url']
    
    try:
        res = requests.get(url, headers=headers)
        res.raise_for_status()
        res.encoding = "utf-8-sig"
        parser = fromstring(res.text)

        title = parser.xpath('//*[@id="print_area"]/div[1]/dl/dt/strong/text()')[0]
        text = parser.xpath('//*[@id="print_area"]/div[2]/div')[0].text_content()

        df.loc[idx, ['제목', '내용']] = [title, text]
    
    except:
        
        try:
            print(f'{idx}번째 url 에러발생!!! 3초를 쉽니다.')
            time.sleep(3)

            res = requests.get(url, headers=headers)
            res.raise_for_status()
            res.encoding = "utf-8-sig"
            parser = fromstring(res.text)

            title = parser.xpath('//*[@id="print_area"]/div[1]/dl/dt/strong/text()')[0]
            text = parser.xpath('//*[@id="print_area"]/div[2]/div')[0].text_content()

            df.loc[idx, ['제목', '내용']] = [title, text]
            
        except:
            
            try:
                
                print(f'{idx}번째 url 2차 에러발생!!! 3초를 쉽니다.')
                time.sleep(3)

                res = requests.get(url, headers=headers)
                res.raise_for_status()
                res.encoding = "utf-8-sig"
                parser = fromstring(res.text)

                title = parser.xpath('//*[@id="print_area"]/div[1]/dl/dt/strong/text()')[0]
                text = parser.xpath('//*[@id="print_area"]/div[2]/div')[0].text_content()

                df.loc[idx, ['제목', '내용']] = [title, text]
                
            except:
                
                try:
                    
                    print(f'{idx}번째 url 3차 에러발생!!! 3초를 쉽니다.')
                    time.sleep(3)

                    res = requests.get(url, headers=headers)
                    res.raise_for_status()
                    res.encoding = "utf-8-sig"
                    parser = fromstring(res.text)

                    title = parser.xpath('//*[@id="print_area"]/div[1]/dl/dt/strong/text()')[0]
                    text = parser.xpath('//*[@id="print_area"]/div[2]/div')[0].text_content()

                    df.loc[idx, ['제목', '내용']] = [title, text]
                    
                except:
                    
                    print(f'=== {idx}번째 url은 삭제된 페이지로 판단되어 수집하지 않습니다. ===')
        
    if idx % 10 == 0:
        print(f'== {idx}개 완료 == 경과시간: {round(time.time() - start)}초 == ')
        
    if idx % 1000 == 0:
        df.to_csv(f'보배드림_베스트글_크롤링_본문까지({idx}까지).csv', index=False) # 중간저장을 하신다면, 경로만 옳게 바꿔줍니다.
        print(f'== {idx}까지 중간저장 완료 ==')